# Attachment Protocol Example - Alice

## Role: 

This notebook tests the capablity to extend the basic controller to control a custom protocol developed in an aries agent. This protocol allows you to attach a file to a message.
Run this along side [Attachment Protocol Bob](http://localhost:8889/notebooks/attachment.ipynb)

In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


## Intialise a controller 

In [2]:
from attachment_controller.attachment_controller import AttachmentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# Based on the aca-py agent you wish to control
attach_controller = AttachmentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

## Confirm that an active connection exists

This can fail sometimes, there are fallback steps described in Bob's notebook.

In [3]:
response = await attach_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':   
        global connection_id
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
    else:
        print("Connection is still progressing to active state, retry in a few moments")
else:
    print("You must create a connection")
    

Results :  [{'request_id': 'c13c92dc-7d8d-4cfe-804d-fd093ccde972', 'connection_id': '6a5a8061-6823-4a4f-9d31-081d1475aa0d', 'routing_state': 'none', 'accept': 'manual', 'invitation_mode': 'once', 'invitation_key': 'JD3qz1iKdZJhVBLka67fedits8mtLNqpgbm1zc4wXhbG', 'updated_at': '2020-10-13 13:01:46.183807Z', 'state': 'active', 'initiator': 'external', 'my_did': 'QZujNVJos7oeGn4MsuQUD', 'created_at': '2020-10-13 13:01:25.959658Z', 'their_did': 'RQmiPRs2oUhybgZvY8U7JG', 'their_label': 'Bob'}]
Connection : {'request_id': 'c13c92dc-7d8d-4cfe-804d-fd093ccde972', 'connection_id': '6a5a8061-6823-4a4f-9d31-081d1475aa0d', 'routing_state': 'none', 'accept': 'manual', 'invitation_mode': 'once', 'invitation_key': 'JD3qz1iKdZJhVBLka67fedits8mtLNqpgbm1zc4wXhbG', 'updated_at': '2020-10-13 13:01:46.183807Z', 'state': 'active', 'initiator': 'external', 'my_did': 'QZujNVJos7oeGn4MsuQUD', 'created_at': '2020-10-13 13:01:25.959658Z', 'their_did': 'RQmiPRs2oUhybgZvY8U7JG', 'their_label': 'Bob'}
Active Connect

In [4]:
print(connection_id)

019bf12b-4f57-4f2d-b3ad-8564f1ea1846


### Note: We do not need to initialise any listeners for Alice. In this example she only sends attachments and does not receive them

## Make sure you have initialised the handler on [Bob's notebook](http://localhost:8889/notebooks/attachment.ipynb)

## Send an attachment over DIDcomm to Bob

In this instance a text file is sent, this will be received and saved by the handler in the [Bob notebook](http://localhost:8889/notebooks/attachment.ipynb).

In [4]:
from aiohttp import FormData
data = FormData()
data.add_field('file',
               open('test_file.txt', 'rb'),
               filename='test_file.txt',
               content_type='text/plain')
response = await attach_controller.protocol.send_attachment(connection_id, data)
print('Attachment- Alice-> Bob')
print(response)


TypeError: Object of type FormData is not JSON serializable

Here an image file is sent and is received by the handler in [Bob notebook](http://localhost:8889/notebooks/attachment.ipynb).

In [15]:
data = FormData()
data.add_field('file',
               open('openmined.jpg', 'rb'),
               filename='openmined.jpg',
               content_type='image/jpeg')
response = await attach_controller.protocol.send_attachment(connection_id, data)
print('Attachment- Alice-> Bob')
print(response)

Attachment- Alice-> Bob
{'thread_id': '4b541001-77ea-49b2-85ef-7c7bf9f06ceb'}



# End of Tutorial

Be sure to terminate the controller so you can run another tutorial

In [8]:
response = await attach_controller.terminate()
print(response)


None
